In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install mediapipe tensorflow opencv-python numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing inst

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


In [18]:
import numpy as np

# ✅ Your sign categories
actions = np.array(['Good Morning', 'Give', 'Crocodile', 'Maybe', 'Knife'])

# Save this list so we use the same one later (important!)
np.save("/content/drive/MyDrive/isl_models/actions.npy", actions)

print("🎯 Actions saved for reference:", actions)


🎯 Actions saved for reference: ['Good Morning' 'Give' 'Crocodile' 'Maybe' 'Knife']


In [19]:
mp_holistic = mp.solutions.holistic

def extract_keypoints_from_frame(results):
    pose, lh, rh = [], [], []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            pose.extend([lm.x, lm.y, lm.z, lm.visibility])
    else:
        pose = [0] * 132

    if results.left_hand_landmarks:
        for lm in results.left_hand_landmarks.landmark:
            lh.extend([lm.x, lm.y, lm.z])
    else:
        lh = [0] * 63

    if results.right_hand_landmarks:
        for lm in results.right_hand_landmarks.landmark:
            rh.extend([lm.x, lm.y, lm.z])
    else:
        rh = [0] * 63

    return np.array(pose + lh + rh)

def video_to_keypoints(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    sequence = []
    with mp_holistic.Holistic(min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            keypoints = extract_keypoints_from_frame(results)
            sequence.append(keypoints)
            if len(sequence) == max_frames:
                break
        cap.release()
    # pad sequence if shorter than 30 frames
    if len(sequence) < max_frames:
        sequence.extend([np.zeros(258)] * (max_frames - len(sequence)))
    return np.array(sequence)


In [22]:
video_path = "/content/drive/MyDrive/kaggle_datasets/indian_sign_language/Video_Dataset/Video_Dataset/Good Morning/WIN_20231103_13_50_28_Pro.mp4"

# Preprocess video
sequence = video_to_keypoints(video_path)
input_data = np.expand_dims(sequence, axis=0)  # shape (1, 30, 258)

# Predict
prediction = model.predict(input_data)
predicted_index = np.argmax(prediction)
predicted_word = actions[predicted_index]

print("🧠 Predicted word:", predicted_word)
print("🔢 Raw probabilities:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
🧠 Predicted word: Good Morning
🔢 Raw probabilities: [[4.8138896e-01 4.3456617e-01 6.4530033e-03 7.7519581e-02 7.2177354e-05]]


In [23]:
top_indices = prediction[0].argsort()[-3:][::-1]
for i in top_indices:
    print(f"{actions[i]} → {prediction[0][i]*100:.2f}%")


Good Morning → 48.14%
Give → 43.46%
Maybe → 7.75%
